Pleaes run preprocess_Data module first before run this module.

Author: xzhang867

## Install Packages

In [ ]:
!pip install visualkeras
!pip install keras_sequential_ascii

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Packages

In [ ]:
import pandas as pd
import visualkeras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras_sequential_ascii import keras2ascii

from google.colab import drive

## Map Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Build CNN Model

### Set Common Parameters



In [ ]:
nb_train_samples =2312
nb_validation_samples = 663
epochs = 10
img_width, img_height = 256, 256
batch_size = 16

### Generate Training Dataset

In [ ]:
train_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/train'

train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 2312 images belonging to 5 classes.


### Generate Validation Dataset

In [ ]:
validation_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/validation'

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 663 images belonging to 5 classes.


### Generate Test Dataset

In [ ]:
test_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/test'

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 328 images belonging to 5 classes.


### Prepare CNN Model

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# initiate CNN model
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add layer
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

# add dropout to avoid overfitting
model.add(Dropout(0.5))

# output layer contains 5 neuron
model.add(Dense(5))
model.add(Activation('softmax'))

### Train the Model

In [ ]:
model.compile(loss='categorical_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

In [ ]:
model.fit(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
144/144 [==============================] - 1436s 10s/step - loss: 1.7033 - accuracy: 0.4608 - val_loss: 1.0861 - val_accuracy: 0.6418
Epoch 2/10
144/144 [==============================] - 136s 942ms/step - loss: 1.0513 - accuracy: 0.6019 - val_loss: 0.7950 - val_accuracy: 0.6890
Epoch 3/10
144/144 [==============================] - 132s 918ms/step - loss: 0.9000 - accuracy: 0.6773 - val_loss: 0.7462 - val_accuracy: 0.7210
Epoch 4/10
144/144 [==============================] - 132s 918ms/step - loss: 0.7965 - accuracy: 0.7030 - val_loss: 0.6684 - val_accuracy: 0.7576
Epoch 5/10
144/144 [==============================] - 135s 933ms/step - loss: 0.7601 - accuracy: 0.7104 - val_loss: 0.7607 - val_accuracy: 0.7348
Epoch 6/10
144/144 [==============================] - 133s 921ms/step - loss: 0.7083 - accuracy: 0.7330 - val_loss: 0.6473 - val_accuracy: 0.7576
Epoch 7/10
144/144 [==============================] - 132s 912ms/step - loss: 0.6850 - accuracy: 0.7483 - val_loss: 0.7387 - 

## Evaluate Model (Test Data)

In [ ]:
evaluate = model.evaluate(test_generator)
print(evaluate)

21/21 [==============================] - 167s 8s/step - loss: 0.6453 - accuracy: 0.7409
[0.645302414894104, 0.7408536672592163]


## Save Model

In [ ]:
model.save('model_saved.h5')

## Visualize the Model

In [ ]:
visualkeras.layered_view(model, to_file='model_output.png').show()

In [ ]:
keras2ascii(model)

           OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

               Input   #####    256  256    3
              Conv2D    \|/  -------------------       416     0.0%
                relu   #####    255  255   32
        MaxPooling2D   Y max -------------------         0     0.0%
                       #####    127  127   32
              Conv2D    \|/  -------------------      4128     0.1%
                relu   #####    126  126   32
        MaxPooling2D   Y max -------------------         0     0.0%
                       #####     63   63   32
              Conv2D    \|/  -------------------      8256     0.1%
                relu   #####     62   62   64
        MaxPooling2D   Y max -------------------         0     0.0%
                       #####     31   31   64
             Flatten   ||||| -------------------         0     0.0%
                       #####       61504
               Dense   XXXXX -------------------   7872640    99.8%
                rel